In [11]:
import string
import random
import pandas as pd
from typing import List,Iterator,Tuple
from pyspark.sql.pandas.functions import pandas_udf
from pyspark.sql.functions import struct, col

from pyspark.sql.types import StringType,DoubleType
from sparkstudy.deploy.demo_sessions import DemoSQLSessionFactory
%load_ext autoreload
%autoreload 2
%matplotlib inline

COLUMNS = ["name","age","salary"]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


比较下开不开启arrow的区别

测试下来，感觉性能提升有点奇怪。有时候会快，有时候会慢。

In [12]:
def create_random_data(row_num:int)->List[tuple]:
     result = list()
     a_str = string.ascii_uppercase
     for i in range(row_num):
         random_letter = random.choice(a_str)
         result.append((random_letter,random.randint(1,row_num),random.random()))
     return result

In [13]:
def test_performance(session_factory:DemoSQLSessionFactory, n:int = 100000):
    data = create_random_data(n)
    spark_session = session_factory.build_session()
    df = spark_session.createDataFrame(data,COLUMNS).cache()
    df.toPandas().head(5)

In [14]:
session_factory_normal = DemoSQLSessionFactory(name="normal")
%time test_performance(session_factory_normal)

Wall time: 3.33 s


In [15]:
session_factory_arrow = DemoSQLSessionFactory(name="with arraw")
session_factory_arrow.add_config("spark.sql.execution.arrow.pyspark.enabled","true")
%time test_performance(session_factory_arrow)

Wall time: 3.01 s


常规的HelloWorld的example。
页面上面的第一个例子。本质就是生成一个新的dataframe
1. 在annotation上面列出的是新的dataframe的col和类型
2. 他会自动的把pd的转换成spark的
3. 函数应该会分批node执行。然后再汇总。因为我看到了。hello world的函数会被执行好几次

In [16]:
session_factory_arrow.add_config('spark.sql.execution.arrow.maxRecordsPerBatch',10)
spark = session_factory_arrow.build_session()
test_data = create_random_data(row_num=1000)
basic_df = spark.createDataFrame(test_data,COLUMNS)
basic_df.show()

+----+---+-------------------+
|name|age|             salary|
+----+---+-------------------+
|   B| 53|0.07222414671306154|
|   S|284| 0.7591491065150204|
|   W|795| 0.1321503938480999|
|   H|817|0.23527904716797643|
|   P|391|   0.56607862908039|
|   D|517| 0.9009097520324495|
|   C|657| 0.6629804651945165|
|   U|427| 0.5658856496492505|
|   D|702| 0.8122834294088883|
|   O| 53| 0.9928525356156671|
|   F|612| 0.4957293923077204|
|   C|716| 0.4814493693803298|
|   L|279|0.23051910331483128|
|   Y|626|0.03737641069339437|
|   D|965|     0.981804320733|
|   X|756|  0.729066895773649|
|   T|430| 0.4098659049818115|
|   J|565|0.38979064683209386|
|   T|751| 0.4876784057142557|
|   H| 35| 0.3326300009380576|
+----+---+-------------------+
only showing top 20 rows



In [17]:
@pandas_udf("total double")
def func(s1: pd.Series, s2: pd.Series) -> pd.DataFrame:
    print("execute")
    s3 = pd.DataFrame()
    s3['total'] = s1 + s2
    return s3
basic_df.select(func("age","salary").alias("result")).show()

+--------------------+
|              result|
+--------------------+
| [53.07222414671306]|
|  [284.759149106515]|
| [795.1321503938481]|
|  [817.235279047168]|
| [391.5660786290804]|
| [517.9009097520325]|
| [657.6629804651946]|
|[427.56588564964926]|
| [702.8122834294089]|
| [53.99285253561567]|
| [612.4957293923077]|
| [716.4814493693804]|
| [279.2305191033148]|
| [626.0373764106934]|
|  [965.981804320733]|
| [756.7290668957736]|
| [430.4098659049818]|
| [565.3897906468321]|
| [751.4876784057143]|
| [35.33263000093806]|
+--------------------+
only showing top 20 rows



主要是想要看看。select方法，不能不能接受一个List

In [18]:
def to_str_func(s1: pd.Series) -> pd.Series:
    return s1.astype(dtype=str)
to_str = pandas_udf(to_str_func, returnType=StringType())

age_c = to_str("age").alias("age")
salary_c = to_str("salary").alias("salary")
selects = [age_c,salary_c]
basic_df.select(selects).show()

+---+-------------------+
|age|             salary|
+---+-------------------+
| 53|0.07222414671306154|
|284| 0.7591491065150204|
|795| 0.1321503938480999|
|817|0.23527904716797643|
|391|   0.56607862908039|
|517| 0.9009097520324495|
|657| 0.6629804651945165|
|427| 0.5658856496492505|
|702| 0.8122834294088883|
| 53| 0.9928525356156671|
|612| 0.4957293923077204|
|716| 0.4814493693803298|
|279|0.23051910331483128|
|626|0.03737641069339437|
|965|     0.981804320733|
|756|  0.729066895773649|
|430| 0.4098659049818115|
|565|0.38979064683209386|
|751| 0.4876784057142557|
| 35| 0.3326300009380576|
+---+-------------------+
only showing top 20 rows



测试以下。如果参数是不定的行不行

简单的来书，
- 确定的column个数，用Series
- 不确定用dataframe
- iterator是类似用流

In [19]:
@pandas_udf("double")
def to_sum_func(data: pd.DataFrame) -> pd.Series:
    return data.age*data.salary
cols = [col("age"),col("salary")]
headers = struct(cols)
#my_sum = pandas_udf(to_sum_func, returnType=DoubleType())
basic_df.select(to_sum_func(headers).alias("result")).show()


+------------------+
|            result|
+------------------+
| 3.827879775792262|
| 215.5983462502658|
|105.05956310923942|
|192.22298153623674|
|221.33674397043248|
| 465.7703418007764|
| 435.5781656327973|
|241.63317240022997|
| 570.2229674450396|
| 52.62118438763036|
|303.38638809232486|
| 344.7177484763161|
| 64.31482982483793|
|23.397633094064876|
|  947.441169507345|
| 551.1745732048787|
|176.24233914217896|
|220.23171546013302|
|  366.246482691406|
|11.642050032832017|
+------------------+
only showing top 20 rows



能不能用于SQL

In [ ]:
basic_df.createOrReplaceTempView("pandas_udf")
spark.udf.register("pandas_to_str", to_str)
spark.sql("select pandas_to_str(age) from pandas_udf").show()

basic_df.createOrReplaceTempView("pandas_udf")
spark.udf.register("pandas_to_str", to_str)
spark.sql("select pandas_to_str(age) from pandas_udf").show()
